In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
gc.collect()

0

### Read the Input CSV files

In [3]:
DiaP = pd.read_csv(".\csv_files\DiaP.csv",low_memory=False,dtype=str,encoding="unicode_escape",skiprows=3,header= 0)

In [4]:
DiaE = pd.read_csv(".\csv_files\DiaE.csv",low_memory=False,dtype=str,encoding="unicode_escape",skiprows=3,header= 0)

In [5]:
Export = pd.read_csv(".\csv_files\Export.csv",low_memory=False,dtype=str,encoding="unicode_escape")

### Replace nan 

In [6]:
Export.replace(np.nan, "", inplace=True)
DiaP.replace(np.nan, "", inplace=True)
DiaE.replace(np.nan, "", inplace=True)

### Initialize DiaP

In [7]:
%%time
DiaP["ID"] = ""
for row in range(len(DiaP.index)):
    DiaP["ID"][row] = f'{DiaP["Teil"][row]}{DiaP["Submodul"][row]}{DiaP["POS"][row]}{DiaP["Codebedingung"][row]}{DiaP["AA"][row]}'

CPU times: total: 4min
Wall time: 5min 8s


### Initialize DiaE

In [8]:
%%time
DiaE["ID FW"] = ""
DiaE["ID FS"] = ""
DiaE["ID FV"] = ""
for row in range(len(DiaE.index)):
    DiaE["ID FW"][row] = f'{DiaE["Teil"][row]}{DiaE["Submodul"][row]}{DiaE["POS"][row]}{DiaE["Codebedingung"][row]}{DiaE["FW MG"][row]}'
    DiaE["ID FS"][row] = f'{DiaE["Teil"][row]}{DiaE["Submodul"][row]}{DiaE["POS"][row]}{DiaE["Codebedingung"][row]}{DiaE["FS MG"][row]}'
    DiaE["ID FV"][row] = f'{DiaE["Teil"][row]}{DiaE["Submodul"][row]}{DiaE["POS"][row]}{DiaE["Codebedingung"][row]}{DiaE["FV MG"][row]}'

CPU times: total: 5min 40s
Wall time: 5min 41s


### Initialize Export

In [9]:
%%time
Export["KEM AGG"] = ""
Export["ID"] = ""
Export["KEM NEU VEW FW"] = ""
Export["KEM NEU VEW FS"] = ""
Export["KEM NEU VEW FV"] = ""
Export["KEM UNG"] = "-"
Export["PEM NEU VEW"] = ""
Export["PEM UNG"] = ""
Export["EIN"] = ""
Export["AUS"] = ""
Export["Comment"] = ""
for row in range(len(Export.index)):
    if Export["KEM_kems"][row] == Export["KEM_kemv"][row]:
        Export["KEM AGG"][row] = f'{Export["KEM_kems"][row]}'
    else:
        Export["KEM AGG"][row] = f'{Export["KEM_kems"][row]}{Export["KEM_kemv"][row]}'
    Export["ID"][row] = f'{Export["Sachnummer_snrb"][row]}{Export["Submodul_sollv"][row]}{Export["POS_sollv"][row]}{Export["Code_sollv"][row]}{Export["AA_sollv"][row]}'

CPU times: total: 1min 1s
Wall time: 1min 1s


### Calculation part for Export, DiaP

In [10]:
temp = pd.DataFrame()
temp["ID"] = DiaP["ID"]
temp["PEM ab"] = DiaP["PEM ab"]
temp["PEM bis"] = DiaP["PEM bis"]
temp["Termin ab"] = DiaP["Termin ab"]
temp["Termin bis"] = DiaP["Termin bis"]
temp.drop_duplicates(subset="ID", keep="last", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
Export["PEM NEU VEW"] = Export["PEM ab"]
Export["PEM UNG"] = Export["PEM bis"]
Export["EIN"] = Export["Termin ab"]
Export["AUS"] = Export["Termin bis"]
del Export["PEM ab"]
del Export["PEM bis"]
del Export["Termin bis"]
del Export["Termin ab"]
del temp

### Calculation part for Export,DiaE

#### for fv

In [11]:
temp = pd.DataFrame()
temp["ID"] = DiaE["ID FV"]
temp["KEM ab"] = DiaE["KEM ab"]
temp.drop_duplicates(subset="ID", keep="first", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
Export["KEM NEU VEW FV"] = Export["KEM ab"]
del Export["KEM ab"]
del temp

In [12]:
temp = pd.DataFrame()
temp["ID"] = DiaE["ID FV"]
temp["KEM bis"] = DiaE["KEM bis"]
temp.drop_duplicates(subset="ID", keep="last", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
for row in range(len(Export.index)):
    if Export["KEM UNG"][row] == "-":
        Export["KEM UNG"][row] = Export["KEM bis"][row]
Export["KEM UNG"].replace("", "-",inplace=True)
del Export["KEM bis"]
del temp

#### for fw

In [13]:
temp = pd.DataFrame()
temp["ID"] = DiaE["ID FW"]
temp["KEM ab"] = DiaE["KEM ab"]
temp.drop_duplicates(subset="ID", keep="first", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
Export["KEM NEU VEW FW"] = Export["KEM ab"]
del Export["KEM ab"]
del temp

In [14]:
temp = pd.DataFrame()
temp["ID"] = DiaE["ID FW"]
temp["KEM bis"] = DiaE["KEM bis"]
temp.drop_duplicates(subset="ID", keep="last", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
for row in range(len(Export.index)):
    if Export["KEM UNG"][row] == "-":
        Export["KEM UNG"][row] = Export["KEM bis"][row]
Export["KEM UNG"].replace("", "-",inplace=True)
del Export["KEM bis"]
del temp

#### for fs

In [15]:
temp = pd.DataFrame()
temp["ID"] = DiaE["ID FS"]
temp["KEM ab"] = DiaE["KEM ab"]
temp.drop_duplicates(subset="ID", keep="first", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
Export["KEM NEU VEW FS"] = Export["KEM ab"]
del Export["KEM ab"]
del temp

In [16]:
temp = pd.DataFrame()
temp["ID"] = DiaE["ID FS"]
temp["KEM bis"] = DiaE["KEM bis"]
temp.drop_duplicates(subset="ID", keep="last", ignore_index=True, inplace=True)
Export = Export.merge(temp, on="ID", how="left")
Export.replace(np.nan, "", inplace=True)
for row in range(len(Export.index)):
    if Export["KEM UNG"][row] == "-":
        Export["KEM UNG"][row] = Export["KEM bis"][row]
Export["KEM UNG"].replace("", "-",inplace=True)
del Export["KEM bis"]
del temp

### Get the output

In [17]:
Export.to_csv("./Export_out.csv", index=False)